In [ ]:
import glob
import pandas as pd

# 1. Build a mapping for the columns with mixed types
dtype_map = {
    'Day'               :'string',
    'timezone'          :'string',
    'truck'             :'string',
    'Role'              :'string',
    'Transfer unit'     :'string',
    'bitrate_unit'      :'string',
    'cwnd_unit'         :'string',
    'Role-RX'           :'string',
    'Transfer unit-RX'  :'string',
    'bitrate_unit-RX'   :'string',
    'square_id'         :'string',
}

# 2. Reload all files with low_memory=False
file_paths = glob.glob('data/*-combined-kml.csv')
df = pd.concat([
    pd.read_csv(p, dtype=dtype_map, low_memory=False)
    for p in file_paths
], ignore_index=True)


numeric_cols = [
    'time','Year','Month','Date','hour','min','sec',
    'latitude','longitude','speed',
    'svr1','svr2','svr3','svr4',
    'Transfer size','Bitrate','Retransmissions','CWnd',
    'Transfer size-RX','Bitrate-RX','send_data'
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

print(df.dtypes)


time                         int64
Day                 string[python]
Year                       float64
Month                      float64
Date                       float64
hour                       float64
min                        float64
sec                        float64
timezone            string[python]
latitude                   float64
longitude                  float64
speed                      float64
truck               string[python]
svr1                       float64
svr2                       float64
svr3                       float64
svr4                       float64
Role                string[python]
Transfer size              float64
Transfer unit       string[python]
Bitrate                    float64
bitrate_unit        string[python]
Retransmissions            float64
CWnd                       float64
cwnd_unit           string[python]
Role-RX             string[python]
Transfer size-RX           float64
Transfer unit-RX    string[python]
Bitrate-RX          

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3042495 entries, 0 to 3042494
Data columns (total 32 columns):
 #   Column            Dtype  
---  ------            -----  
 0   time              int64  
 1   Day               string 
 2   Year              float64
 3   Month             float64
 4   Date              float64
 5   hour              float64
 6   min               float64
 7   sec               float64
 8   timezone          string 
 9   latitude          float64
 10  longitude         float64
 11  speed             float64
 12  truck             string 
 13  svr1              float64
 14  svr2              float64
 15  svr3              float64
 16  svr4              float64
 17  Role              string 
 18  Transfer size     float64
 19  Transfer unit     string 
 20  Bitrate           float64
 21  bitrate_unit      string 
 22  Retransmissions   float64
 23  CWnd              float64
 24  cwnd_unit         string 
 25  Role-RX           string 
 26  Transfer size-

,time,Day,Year,Month,Date,hour,min,sec,timezone,latitude,...,Retransmissions,CWnd,cwnd_unit,Role-RX,Transfer size-RX,Transfer unit-RX,Bitrate-RX,bitrate_unit-RX,send_data,square_id
0,1656876656,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
1,1656876656,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
2,1656876656,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
3,1656876657,Mon,2022.0,7.0,4.0,5.0,30.0,57.0,AEST,99.0,...,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,0.0,<NA>
4,1656876657,Mon,2022.0,7.0,4.0,5.0,30.0,57.0,AEST,99.0,...,NaN,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,0.0,<NA>


In [3]:
df_copy = df.copy()

In [18]:
df = df_copy.copy()

In [19]:
df['datetime'] = pd.to_datetime(df['time'], unit='s')
for c in ['latitude', 'longitude']:
    df.loc[df[c] == 999, c] = pd.NA
df.dropna(subset=['latitude', 'longitude'], inplace=True)

df.datetime = df.datetime.dt.tz_localize('UTC').dt.tz_convert('Australia/Melbourne')

In [20]:
# Define the column groups
svr_cols = ['svr1', 'svr2', 'svr3', 'svr4']
other_cols = ['Transfer size', 'Bitrate', 'Retransmissions', 'CWnd',
              'Transfer size-RX', 'Bitrate-RX', 'send_data']

# Filter rows where svr1 to svr4 are all 1000 and the rest are all 0
filtered_df = df[
    (df[svr_cols].eq(1000).all(axis=1)) & 
    (df[other_cols].eq(0).all(axis=1))
]

# Drop the filtered rows from the original DataFrame
df = df.drop(filtered_df.index)

# Check the resulting DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2315616 entries, 837 to 3037653
Data columns (total 33 columns):
 #   Column            Dtype                              
---  ------            -----                              
 0   time              int64                              
 1   Day               string                             
 2   Year              float64                            
 3   Month             float64                            
 4   Date              float64                            
 5   hour              float64                            
 6   min               float64                            
 7   sec               float64                            
 8   timezone          string                             
 9   latitude          float64                            
 10  longitude         float64                            
 11  speed             float64                            
 12  truck             string                             
 13  

In [21]:
to_drop = ['time','Year', 'timezone', 'sec', 'Role', 'Month', 'Role-RX' , 'Transfer unit-RX', 'cwnd_unit',  'bitrate_unit', 'Transfer unit', 'bitrate_unit-RX', 'Day']

df.drop(columns=to_drop, inplace=True)

In [22]:
df.set_index('datetime', inplace=True)

In [23]:
df.dropna(how='any', inplace=True)

In [24]:
df.reset_index()

,datetime,Date,hour,min,latitude,longitude,speed,truck,svr1,svr2,svr3,svr4,Transfer size,Bitrate,Retransmissions,CWnd,Transfer size-RX,Bitrate-RX,send_data,square_id
0,2022-07-04 05:43:37+10:00,4.0,5.0,43.0,-37.737985,144.849691,76.811700,garbo02,25.5,28.9,27.8,44.5,0.864258,7.25,0.0,0.007080,0.988281,8.29,0.000000,square_94489280583
1,2022-07-04 05:43:38+10:00,4.0,5.0,43.0,-37.738127,144.849794,75.519004,garbo02,35.0,32.0,31.3,42.1,1.300000,10.90,3.0,0.010645,0.972656,8.16,0.000000,square_94489280583
2,2022-07-04 05:43:39+10:00,4.0,5.0,43.0,-37.738249,144.849904,72.846568,garbo02,34.2,30.5,29.8,46.1,1.300000,10.90,0.0,0.010645,0.957031,8.03,0.000000,square_94489280583
3,2022-07-04 05:43:40+10:00,4.0,5.0,43.0,-37.738362,144.850020,70.279696,garbo02,25.0,23.2,21.6,38.7,0.864258,7.25,0.0,0.007080,0.953125,7.99,0.014404,square_94489280583
4,2022-07-04 05:43:41+10:00,4.0,5.0,43.0,-37.738491,144.850142,70.153760,garbo02,20.2,18.7,23.4,34.2,0.926758,7.77,6.0,0.007588,0.927734,7.78,0.000000,square_94489280583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707957,2022-07-07 10:21:57+10:00,7.0,10.0,21.0,-37.745164,144.809922,5.393024,garbo06,304.0,308.0,301.0,324.0,1.250000,10.50,1.0,0.010254,2.080000,17.40,0.000000,square_85899346000
1707958,2022-07-07 10:21:58+10:00,7.0,10.0,21.0,-37.745170,144.809933,4.978176,garbo06,232.0,230.0,228.0,245.0,1.250000,10.50,0.0,0.010254,1.440000,12.10,0.000000,square_85899346000
1707959,2022-07-07 10:22:00+10:00,7.0,10.0,22.0,-37.745228,144.809942,8.911824,garbo06,168.0,166.0,159.0,190.0,1.250000,10.50,4.0,0.010254,1.540000,12.90,0.014404,square_85899346000
1707960,2022-07-07 10:22:01+10:00,7.0,10.0,22.0,-37.745238,144.809933,7.274656,garbo06,281.0,272.0,271.0,288.0,2.500000,21.00,0.0,0.020508,1.610000,13.50,0.000000,square_85899346000


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1707962 entries, 2022-07-04 05:43:37+10:00 to 2022-07-07 10:22:02+10:00
Data columns (total 19 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Date              float64
 1   hour              float64
 2   min               float64
 3   latitude          float64
 4   longitude         float64
 5   speed             float64
 6   truck             string 
 7   svr1              float64
 8   svr2              float64
 9   svr3              float64
 10  svr4              float64
 11  Transfer size     float64
 12  Bitrate           float64
 13  Retransmissions   float64
 14  CWnd              float64
 15  Transfer size-RX  float64
 16  Bitrate-RX        float64
 17  send_data         float64
 18  square_id         string 
dtypes: float64(17), string(2)
memory usage: 260.6 MB


In [26]:
df.to_csv('Cleaned.csv')

In [ ]:

# this will one‑hot encode both columns and append the new dummy columns
df_encoded = pd.get_dummies(
    df,
    columns=['square_id', 'truck'],
    prefix=['square', 'truck'],
    drop_first=False,      
    dtype='uint16'            # smaller integer dtype to save memory
)

# inspect the new column names
print(df_encoded.columns.tolist())


['Date', 'hour', 'min', 'latitude', 'longitude', 'speed', 'svr1', 'svr2', 'svr3', 'svr4', 'Transfer size', 'Bitrate', 'Retransmissions', 'CWnd', 'Transfer size-RX', 'Bitrate-RX', 'send_data', 'square_square_103079215126', 'square_square_103079215128', 'square_square_103079215130', 'square_square_103079215132', 'square_square_103079215134', 'square_square_103079215136', 'square_square_103079215138', 'square_square_103079215140', 'square_square_103079215142', 'square_square_103079215144', 'square_square_103079215147', 'square_square_103079215151', 'square_square_103079215153', 'square_square_103079215155', 'square_square_103079215157', 'square_square_103079215158', 'square_square_103079215180', 'square_square_103079215182', 'square_square_103079215184', 'square_square_103079215186', 'square_square_103079215188', 'square_square_111669149696', 'square_square_111669149698', 'square_square_111669149700', 'square_square_111669149702', 'square_square_111669149706', 'square_square_111669149708'

In [28]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1707962 entries, 2022-07-04 05:43:37+10:00 to 2022-07-07 10:22:02+10:00
Columns: 318 entries, Date to truck_garbo11
dtypes: float64(17), uint16(301)
memory usage: 1.2 GB


In [29]:
df_encoded.to_csv('Cleaned_encoded.csv')